In [1]:
import numpy as np
import scipy.stats as stats


In [2]:
N = 10  # Kapasitas stok maksimum
harga_jual = 5  # Harga jual per unit
biaya_pemesanan = 2  # Biaya pemesanan per unit
biaya_produksi = 3  # Biaya produksi per unit
penalti_kehabisan = 4  # Penalti kehilangan penjualan per unit
rata_rata_permintaan = 5  # Rata-rata permintaan per hari (untuk distribusi Poisson)


In [3]:
# Distribusi Poisson untuk permintaan harian
def distribusi_permintaan(lamda):
    return stats.poisson.pmf(np.arange(N+1), lamda)



In [4]:
def reward(stok_awal, aksi, permintaan):
    stok_akhir = max(0, stok_awal + aksi - permintaan)
    penjualan = min(stok_awal + aksi, permintaan)  # Unit yang terjual
    biaya = aksi * biaya_pemesanan  # Biaya pemesanan
    penalti = max(0, permintaan - stok_akhir) * penalti_kehabisan  # Penalti jika kehabisan stok
    return (penjualan * (harga_jual - biaya_produksi)) - biaya - penalti


In [8]:
# Definisi fungsi value_iteration
def value_iteration(gamma=0.9, theta=0.0001):
    V = np.zeros(N+1)  # Nilai inisial untuk setiap status stok
    policy = np.zeros(N+1, dtype=int)  # Kebijakan inisial (jumlah stok yang dipesan untuk setiap status)
    
    while True:
        delta = 0
        for s in range(N+1):
            v = V[s]
            action_values = []
            for a in range(N+1 - s):
                expected_value = 0
                for demand in range(N+1):
                    prob = distribusi_permintaan(rata_rata_permintaan)[demand]
                    next_state = max(0, s + a - demand)
                    reward_value = reward(s, a, demand)
                    expected_value += prob * (reward_value + gamma * V[next_state])
                action_values.append(expected_value)
            V[s] = max(action_values)
            policy[s] = np.argmax(action_values)
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    return V, policy

# Menjalankan Value Iteration
V, policy = value_iteration()

# Menampilkan kebijakan optimal (jumlah stok yang perlu dipesan untuk setiap status)
print("Kebijakan Optimal (Jumlah Stok yang Dipesan per Status):")
for i in range(N+1):
    print(f"Stok {i}: Pesan {policy[i]} unit")

# Menampilkan nilai (value) untuk setiap status stok
print("\nNilai untuk Setiap Status Stok:")
for i in range(N+1):
    print(f"Stok {i}: Nilai = {V[i]}")

Kebijakan Optimal (Jumlah Stok yang Dipesan per Status):
Stok 0: Pesan 10 unit
Stok 1: Pesan 9 unit
Stok 2: Pesan 8 unit
Stok 3: Pesan 7 unit
Stok 4: Pesan 6 unit
Stok 5: Pesan 5 unit
Stok 6: Pesan 4 unit
Stok 7: Pesan 3 unit
Stok 8: Pesan 2 unit
Stok 9: Pesan 1 unit
Stok 10: Pesan 0 unit

Nilai untuk Setiap Status Stok:
Stok 0: Nilai = -66.07198398704296
Stok 1: Nilai = -64.09937588177922
Stok 2: Nilai = -62.12676912383942
Stok 3: Nilai = -60.15416449315267
Stok 4: Nilai = -58.18156265701334
Stok 5: Nilai = -56.20896364013864
Stok 6: Nilai = -54.23636633212119
Stok 7: Nilai = -52.263768581557095
Stok 8: Nilai = -50.291168147398054
Stok 9: Nilai = -48.31856394399277
Stok 10: Nilai = -46.3459565481825
